https://wandb.ai/mukilan/intro_to_gym/reports/A-Gentle-Introduction-to-OpenAI-Gym--VmlldzozMjg5MTA3#:~:text=OpenAI%20Gym%20is%20a%20Pythonic,toolkit%20for%20training%20RL%20algorithms.

In [2]:
import gym

CartPole-v1

CartPole is the Hello World for reinforcement learning. It is a simple environment in which there is a cart with a pole on top of the cart. The cart has some initial velocity, it should learn to adjust its velocity and actions to balance the pole and avoid falling. 

Action Space: Discrete (2); Left or Right

Observation Space: Array with the shape (4). It contains 4 signals:

Cart Position

Cart Velocity

Pole Angle

Pole Angular Velocity


Reward Function: +1 for every time step the pole is standing. 

Termination: When the episode reward exceeds 475, the episode is terminated. 

We'll set the configuration dictionary with the environment name, type of policy, and total time steps. We'll pass this config dictionary to our run which is initialized with wandb.init(). It will automatically log all the necessary metrics. We'll also pass in monitor_gym = True to auto-upload training videos.


In [3]:
env = gym.make("CartPole-v1", render_mode = "human")
env.action_space
env.observation_space.shape
env.action_space.sample()

1

In [3]:
env.reset()
#len(env.step(env.action_space.sample()))
print(env.step(1)[0])
print(env.step(1)[1])
print(env.step(1)[2])
print(env.step(1)[3])
print(env.step(1)[4])

[ 0.01463811  0.1482173   0.00112455 -0.33089066]
1.0
False
False


/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


: 

In [4]:
done = True
for step in range(1000):
    if done:
        env.reset()
    state, reward, done, truncated, info = env.step(env.action_space.sample())
    env.render()
env.close()

/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


KeyboardInterrupt: 

: 

In [37]:
from gym.wrappers import FlattenObservation
env = FlattenObservation(env)

In [4]:
import os 
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import BaseCallback

In [39]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [6]:
CHECKPOINT_DIR = 'CartPole/train/'
LOG_DIR = 'CartPole/logs/'

In [41]:
callback = TrainAndLoggingCallback(check_freq=50, save_path=CHECKPOINT_DIR)
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, n_steps=512)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [42]:
model.learn(total_timesteps=10000, callback=callback)

Logging to CartPole/logs/PPO_2


/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 25.4     |
|    ep_rew_mean     | 25.4     |
| time/              |          |
|    fps             | 47       |
|    iterations      | 1        |
|    time_elapsed    | 10       |
|    total_timesteps | 512      |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 23.7          |
|    ep_rew_mean          | 23.7          |
| time/                   |               |
|    fps                  | 47            |
|    iterations           | 2             |
|    time_elapsed         | 21            |
|    total_timesteps      | 1024          |
| train/                  |               |
|    approx_kl            | 3.3993274e-08 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -0.693        |
|    explained_variance   | 0.00239       |


In [11]:
PPO_path = os.path.join('CartPole', 'Saved_Models')
model.save(PPO_path)

In [12]:
from stable_baselines3.common.evaluation import evaluate_policy
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


NameError: name 'run' is not defined

: 

In [5]:
from stable_baselines3.common.env_util import make_vec_env

model = PPO.load('CartPole/train/best_model_4200.zip', env=env)
#Start the game
vec_env = model.get_env()
vec_env = make_vec_env(vec_env)
obs = vec_env.reset()

#print(model.predict(obs))
#Loop through the game
#while True:
#    action, _state = model.predict(obs)
#    state, reward, done, info = env.step(action)
#    env.render()

/home/marcelo.hurtado/anaconda3/lib/python3.11/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


TypeError: 'DummyVecEnv' object is not callable

In [28]:
model = PPO.load('CartPole/train/best_model_5000.zip')
state = env.reset()
print(model.predict(state[0])[1])

None
